In [19]:
import os
os.environ["SERPAPI_API_KEY"] = "..."

In [20]:
os.environ["OPENAI_API_KEY"]="..."

In [2]:
from langchain.llms import OpenAI

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [5]:
llm = OpenAI(temperature=0)

In [6]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [7]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [10]:
agent.run("How many total wickets Ravichandran Ashwin took against West Indies in last test match? Is it prime number?")



> Entering new AgentExecutor chain...
 I need to find out the number of wickets taken by Ashwin and then check if it is a prime number.
Action: Search
Action Input: "Ravichandran Ashwin wickets West Indies test match"
Observation: Ravichandran Ashwin bags 12 wickets in match as India crush West Indies in first Test in Dominica.
Thought: I need to check if 12 is a prime number.
Action: Calculator
Action Input: 12
Observation: Answer: 12
Thought: 12 is not a prime number.
Final Answer: Ravichandran Ashwin took 12 wickets against West Indies in last test match, but it is not a prime number.

> Finished chain.


'Ravichandran Ashwin took 12 wickets against West Indies in last test match, but it is not a prime number.'

## Plan and execute
Plan and execute agents accomplish an objective by first planning what to do, then executing the sub tasks. 

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain

In [12]:
search = SerpAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

In [13]:
model = ChatOpenAI(temperature=0)

In [14]:
planner = load_chat_planner(model)

In [15]:
executor = load_agent_executor(model, tools, verbose=True)

In [16]:
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [18]:
agent.run("How many total wickets Ravichandran Ashwin took against West Indies in last test match? Is it prime number?")



> Entering new PlanAndExecute chain...
steps=[Step(value="Retrieve the data of Ravichandran Ashwin's wickets against West Indies in the last test match."), Step(value='Calculate the total number of wickets taken by Ravichandran Ashwin.'), Step(value='Check if the total number of wickets is a prime number.'), Step(value="Given the above steps taken, respond to the user's original question. \n")]

> Entering new AgentExecutor chain...
{
  "action": "Search",
  "action_input": "Ravichandran Ashwin wickets against West Indies in last test match"
}

> Finished chain.
*****

Step: Retrieve the data of Ravichandran Ashwin's wickets against West Indies in the last test match.

Response: {
  "action": "Search",
  "action_input": "Ravichandran Ashwin wickets against West Indies in last test match"
}

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Calculator",
  "action_input": "12"
}
```

> Entering new LLMMathChain chain...
12```text
12
```
...numexpr.evaluate("12")...

Answ

'The total number of wickets taken by Ravichandran Ashwin against West Indies in the last test match is 12.'